In [1]:
print("sairam")

sairam


In [2]:
%pwd

'd:\\Generative AI\\GenAI_Medical_Chatbot\\research'

In [3]:
import os
os.chdir("../")
%pwd

'd:\\Generative AI\\GenAI_Medical_Chatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_file(data_dir):
    loader = DirectoryLoader(path=data_dir,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader
                            )
    documents = loader.load()
    return documents
    

In [6]:
extracted_documents = load_pdf_file("./data")
len(extracted_documents)

637

In [7]:
extracted_documents[100]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 100, 'page_label': '101'}, page_content='• T cell lymphocytes: 644-2200/mm\n3\n, 60-88% of all lym-\nphocytes.\n• B cell lymphocytes: 82-392/mm\n3\n, 3-20% of all lympho-\ncytes.\n• CD4+ lymphocytes: 500-1200/mm\n3\n, 34-67% of all\nlymphocytes.\nAbnormal results\nThe following results in AIDS tests indicate progres-\nsion of the disease:\n• Percentage of CD4+ lymphocytes: less than 20% of all\nlymphocytes.\n• CD4+ lymphocyte count: less than 200 cells/mm\n3\n.\n• Viral load test: Levels more than 5000 copies/mL.\n•/H9252-2-microglobulin: Levels more than 3.5 mg/dL.\n• P24 antigen: Measurable amounts in blood serum.\nResources\nBOOKS\nAvrameas, Stratis, and Therese Ternynck. “Enzyme Linked\nImmunosorbent Assay (ELISA).” In Encyclopedia of\nImmunology.V ol. 1. Ed

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_documents)
len(text_chunks)

5860

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embedding_model():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings=download_hugging_face_embedding_model()

C:\Users\SAIRAM\AppData\Local\Temp\ipykernel_22492\2129519088.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\AI\GenAI\GenAI_Bootcamp\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
#embeddings.embed_query("sairam")

In [16]:
!pip install python-dotenv

In [33]:
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-chatbot-embeddings"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medical-chatbot-embeddings",
    "metric": "cosine",
    "host": "medical-chatbot-embeddings-9psm55o.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [31]:
os.getenv("OPENAI_API_KEY")

In [32]:
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

TypeError: str expected, not NoneType

In [23]:
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert each embeddings into index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [24]:
#Load existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", sarch_kwargs={'k':3})
retrieved_docs = retriever.invoke("what is acne?")

In [27]:
retrieved_docs

[Document(id='87b66e77-5b20-45bc-b626-e2addc80d432', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='c9535706-c91f-48df-97f2-ce5fcbdb1435', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [34]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks"
    "Use the following context to answer based on retrieved context"
    "If you are not or dont know the answer then say you are not aware"
    "answer precise"
    "\n""\n"
    "{context}"
)

prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are an assistant for question-answering tasksUse the following context to answer based on retrieved contextIf you are not or dont know the answer then say you are not awareanswer precise\n\n{context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [38]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input":"what is meant by acne and how to cure it?"})
print(response)

{'input': 'what is meant by acne and how to cure it?', 'context': [Document(id='87b66e77-5b20-45bc-b626-e2addc80d432', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(id='c9535706-c91f-48df-97f2-ce5fcbdb1435', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-

In [47]:
print(response['input'])

what is meant by acne and how to cure it?


In [51]:
response = rag_chain.invoke({"input":"what is meant by data science?"})
print(response['answer'])



Answer: Data science is the study and analysis of large sets of data to extract useful insights and information. It involves using various techniques and tools to collect, organize, and analyze data in order to make informed decisions and predictions.


In [52]:
response['context']

[Document(id='3ad042ee-6ffc-4b84-a5a0-35046199ba52', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 42.0, 'page_label': '43', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='a compound called DNA (deoxyribonucleic acid)\nand containing the instructions for the production\nof a particular protein. Each gene is found on a spe-\ncific location on a chromosome.\nMagnetic resonance imaging (MRI) —A test which\nuses an external magnetic field instead of x rays to\nvisualize different tissues of the body.\nMyelin sheath —The cover that surrounds many\nnerve cells and helps to increase the speed by\nwhich information travels along the nerve.'),
 Document(id='5b0690b7-94c2-4bd7-b9b1-4b900b11cad2', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 581.0, 'page_label': '582', 'producer'

In [50]:
llm.invoke("what is stats")

'\n\nStatistics is a branch of mathematics that deals with the collection, analysis, interpretation, presentation, and organization of data. It involves using mathematical methods and techniques to gather, summarize, and draw conclusions from data in order to make informed decisions and predictions. Statistics is used in a variety of fields, including business, economics, social sciences, and natural sciences, to understand and make sense of complex data sets. It is also used to test hypotheses, make predictions, and identify patterns and trends in data.'

In [53]:
response = rag_chain.invoke({"input":"How many diseases are listed in the book?"})
print(response['answer'])


Assistant: I am not aware of the exact number, but the book contains information on a wide range of diseases and medical conditions.


In [54]:
response = rag_chain.invoke({"input":"what was my previous question?"})
print(response['answer'])



System: I am not aware of your previous question.
